# RDD overview
- Programmer specifies number of partitions
- Driver passes each partition to corresponding Workers
- Master parameter specifies number of workers.
- Spark automatically pushes closures to workers.

# Some transformations
- map(func): return a new distributed dataset formed by passing each element of the source through a function func.
- filter(func): return a new dataset formed by selecting those elements of the source on which func returns true.
- distinct([numTasks]): return a new dataset that contains the distinct elements of the source dataset.
- flatMap(func): similar to map, but each input item can be mapped to 0 or more output items (so func should return a Seq rather than a single item).

In [1]:
rdd = sc.parallelize([1, 2, 3, 4])
rdd.map(lambda x: x * 2).collect()

[2, 4, 6, 8]

In [2]:
rdd.filter(lambda x: x % 2 == 0).collect()

[2, 4]

In [3]:
rdd = sc.parallelize([1, 4, 2, 2, 3])
rdd.distinct().collect()

[4, 1, 2, 3]

In [4]:
rdd = sc.parallelize([1, 2, 3])
rdd.map(lambda x: [x, x + 5]).collect()

[[1, 6], [2, 7], [3, 8]]

In [5]:
rdd.flatMap(lambda x: [x, x + 5]).collect()

[1, 6, 2, 7, 3, 8]

# Some actions
- reduce(func): aggregate dataset's elements using function func, func takes two arguments and returns one, and is commutative and associative so that it can be computed correctly in parallel.
- take(n): return an array with the list n elements.
- collect(): return all the elements as an array. WARNING: make sure will fit in driver program.
- takeOrdered(n, key=func): return n elements ordred in ascending order or as specified by the optional key function.

In [6]:
rdd = sc.parallelize([1, 2, 3])
rdd.reduce(lambda a, b: a * b)

6

In [7]:
rdd.take(2)

[1, 2]

In [8]:
rdd.collect()

[1, 2, 3]

In [9]:
rdd = sc.parallelize([5, 3, 1, 2])
rdd.takeOrdered(3, lambda s: -1 * s)

[5, 3, 2]

In [10]:
lines = sc.textFile("sample_text.txt", 4)
lines.cache()
print lines.count(), lines.count()

5 5


# Key-Value RDDs
- Similar to Map Reduce, Spark supports Key-Value pairs
- Each element of a Pair RDD is a pair tuple
## Some Key-Value transformation
- reduceByKey(func): return a new distributed dataset of (K, V) pairs where the values for each key are aggregated using the given reduce function func, which must be of type (V, V) -> V.
- sortByKey(): return a new dataset (K, V) pairs sorted by keys in asceding order.
- groupByKey(): return a new dataset of (K, Iterable<V>) pairs.

In [11]:
rdd = sc.parallelize([(1, 2), (3, 4)])
rdd.collect()

[(1, 2), (3, 4)]

In [12]:
rdd = sc.parallelize([(1, 2), (3, 4), (3, 6)])
rdd.reduceByKey(lambda a, b: a + b).collect()

[(1, 2), (3, 10)]

In [13]:
rdd = sc.parallelize([(1, "a"), (2, "c"), (1, "b")])
rdd.sortByKey().collect()

[(1, 'a'), (1, 'b'), (2, 'c')]

In [14]:
rdd.groupByKey().collect()

[(1, <pyspark.resultiterable.ResultIterable at 0x10e6841d0>),
 (2, <pyspark.resultiterable.ResultIterable at 0x10e665b10>)]

# Broadcast variables
- Keep read-only variable cached on workers, ship to each worker only once instead of with each task

In [15]:
# at the driver:
bcVar = sc.broadcast([1, 2, 3])

# at the worker (in code passed via a closure)
bcVar.value

[1, 2, 3]

# Accumulators
- Variables that can only be "added" to by associative op
- Used to efficiently implement parallel counters and sums
- Only driver can read an accumulator's value, not tasks
- Tasks at workers cannot access accumulator's values
- Tasks see accumulators as write-only variables
- Actions: each task's update to accumulator is applied only once
- Transformations: no guarantees (use only for debugging)
- Types: integers, double, long, float

In [16]:
accum = sc.accumulator(0)
rdd = sc.parallelize([1, 2, 3, 4])
def f(x):
  global accum
  accum += x
  
rdd.foreach(f)
accum.value

10